# Notebook 3: EDA y Feature Engineering
## Monitor de Vulnerabilidad Económica - Colombia

**Objetivo**: Realizar análisis exploratorio exhaustivo, ingeniería de características y preparar datos para modelado.

**Dataset**: `../data/processed/geih_2024_ipm_clean.csv`

**Output**: Datasets de entrenamiento y prueba listos para modelado.

---

**Metodología**:
1. Carga y exploración inicial
2. Definición del target
3. EDA profundo (distribuciones, correlaciones, cardinalidad)
4. Feature Engineering (creación de features derivadas)
5. Tratamiento de variables problemáticas
6. Split estratificado
7. Export de datos procesados

---

In [ ]:
# 1. IMPORTAR LIBRERÍAS

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
import warnings
warnings.filterwarnings('ignore')

# Configuración
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("✓ Librerías importadas correctamente")
print(f"  pandas: {pd.__version__}")
print(f"  numpy: {np.__version__}")

In [ ]:
# 2. CARGAR DATASET LIMPIO

print("="*80)
print("CARGA DE DATOS")
print("="*80)

df = pd.read_csv('../data/processed/geih_2024_ipm_clean.csv')

print(f"\n✓ Dataset cargado exitosamente")
print(f"  Registros: {df.shape[0]:,}")
print(f"  Columnas: {df.shape[1]}")
print(f"\nPrimeras filas:")
display(df.head(3))

print(f"\nColumnas disponibles:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i:2d}. {col}")

---

## 🎯 Definición del Target

**Vulnerabilidad Económica**: Población en riesgo de caer en pobreza.

**Criterio**: Ingreso per cápita entre 1x y 1.5x la línea de pobreza.

- **Línea de Pobreza 2024**: $515,000 COP/mes
- **Zona de Vulnerabilidad**: $515,000 - $772,500 COP/mes

**Target binario**:
- `0`: No vulnerable
- `1`: Vulnerable (entre 1x y 1.5x LP)

---

In [ ]:
# 3. DEFINIR TARGET: VULNERABILIDAD ECONÓMICA

print("="*80)
print("DEFINICIÓN DEL TARGET")
print("="*80)

# Constantes
LINEA_POBREZA_2024 = 515000  # COP por persona/mes
FACTOR_VULNERABILIDAD = 1.5

# Usar INGLABO como proxy de ingreso per cápita
df['ingreso_persona'] = df['INGLABO']

print(f"\nVariable de ingreso: INGLABO")
print(f"  Registros totales: {len(df):,}")
print(f"  Con ingreso > 0: {(df['ingreso_persona'] > 0).sum():,}")
print(f"  Sin ingreso (≤0): {(df['ingreso_persona'] <= 0).sum():,}")

# Filtrar solo registros con ingreso válido
df_valid = df[df['ingreso_persona'] > 0].copy()

print(f"\n✓ Registros con ingreso válido: {len(df_valid):,} ({len(df_valid)/len(df)*100:.1f}%)")

# Estadísticas de ingreso
print(f"\nEstadísticas de ingreso:")
print(f"  Media:    ${df_valid['ingreso_persona'].mean():>15,.0f}")
print(f"  Mediana:  ${df_valid['ingreso_persona'].median():>15,.0f}")
print(f"  Mínimo:   ${df_valid['ingreso_persona'].min():>15,.0f}")
print(f"  Máximo:   ${df_valid['ingreso_persona'].max():>15,.0f}")

# Definir límites de vulnerabilidad
limite_inferior = LINEA_POBREZA_2024
limite_superior = LINEA_POBREZA_2024 * FACTOR_VULNERABILIDAD

print(f"\nLímites de vulnerabilidad:")
print(f"  Línea Pobreza (1x):  ${limite_inferior:>12,}")
print(f"  Límite Superior (1.5x): ${limite_superior:>12,}")

# Crear target
df_valid['IS_VULNERABLE'] = (
    (df_valid['ingreso_persona'] >= limite_inferior) & 
    (df_valid['ingreso_persona'] <= limite_superior)
).astype(int)

# Análisis de distribución
print(f"\n{'='*60}")
print("DISTRIBUCIÓN DEL TARGET")
print(f"{'='*60}")

target_counts = df_valid['IS_VULNERABLE'].value_counts().sort_index()
target_pcts = df_valid['IS_VULNERABLE'].value_counts(normalize=True).sort_index() * 100

print(f"\n  Clase 0 (No Vulnerable): {target_counts[0]:>8,} ({target_pcts[0]:>5.2f}%)")
print(f"  Clase 1 (Vulnerable):    {target_counts[1]:>8,} ({target_pcts[1]:>5.2f}%)")

# Calcular desbalanceo
imbalance_ratio = target_counts[0] / target_counts[1]
print(f"\n  Ratio de desbalanceo: {imbalance_ratio:.2f}:1")
print(f"  ⚠️ Dataset DESBALANCEADO - Requiere técnicas especiales")

# Visualizar
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Gráfico 1: Distribución del target
colors = ['lightgreen', 'coral']
axes[0].bar(['No Vulnerable', 'Vulnerable'], target_counts.values, color=colors, edgecolor='black')
axes[0].set_title('Distribución del Target', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Cantidad')
for i, (count, pct) in enumerate(zip(target_counts.values, target_pcts.values)):
    axes[0].text(i, count, f'{count:,}\n({pct:.1f}%)', ha='center', va='bottom', fontsize=11)

# Gráfico 2: Distribución de ingresos por clase
axes[1].hist(df_valid[df_valid['IS_VULNERABLE']==0]['ingreso_persona'], 
             bins=100, alpha=0.6, label='No Vulnerable', color='lightgreen', range=(0, 3000000))
axes[1].hist(df_valid[df_valid['IS_VULNERABLE']==1]['ingreso_persona'], 
             bins=100, alpha=0.6, label='Vulnerable', color='coral', range=(0, 3000000))
axes[1].axvline(limite_inferior, color='red', linestyle='--', linewidth=2, label=f'LP: ${limite_inferior:,}')
axes[1].axvline(limite_superior, color='orange', linestyle='--', linewidth=2, label=f'1.5x LP: ${limite_superior:,}')
axes[1].set_title('Distribución de Ingresos por Clase', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Ingreso (COP)')
axes[1].set_ylabel('Frecuencia')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"\n✓ Target definido y analizado")

---

## 📊 Análisis Exploratorio de Datos (EDA)

Analizaremos:
1. Tipos de variables y cardinalidad
2. Variables problemáticas (alta cardinalidad)
3. Distribuciones de variables numéricas
4. Relación con el target
5. Identificación de features importantes

---
